In [1]:
import pandas as pd
from pathlib import Path

Train

In [2]:
Catholic_file = 'breathdata/augment_train.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class
0,0001-1.wav,wheezing,1
1,0001-2.wav,healthy,0
2,0001-3.wav,wheezing,1
3,0001-4.wav,wheezing,1
4,0002-1.wav,healthy,0
...,...,...,...
1465,minus_0614-3.wav,wheezing,1
1466,minus_0614-4.wav,wheezing,1
1467,minus_0615-1.wav,wheezing,1
1468,minus_0615-2.wav,wheezing,1


In [3]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class']]
df.head()

,relative_path,class
0,/0001-1.wav,1
1,/0001-2.wav,0
2,/0001-3.wav,1
3,/0001-4.wav,1
4,/0002-1.wav,0


In [4]:
data_path = 'breathdata/train'

In [5]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    sig = torchaudio.functional.gain(sig, gain_db=5.0)
    sig = torchaudio.functional.lowpass_biquad(sig, sr, cutoff_freq=3000)
    sig = torchaudio.functional.highpass_biquad(sig, sr, cutoff_freq=2000)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


## 형 여기 time shift도 빼는게 좋지않으까요?? 아 이미 되잇는거니가 2번하게대지??네ㅇㅋ
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [6]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision

class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    
    return aug_sgram, class_id

In [7]:
from torch.utils.data import random_split

brds = breathDS(df, data_path)

In [8]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

# --------------------------------------------------------------
# 호흡음의 Healthy, Wheezing을 판단하는 Binary Classification Model
# --------------------------------------------------------------

class WhoWheezing(nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=64, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x

Model1 = WhoWheezing()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [9]:
# using the example from the README
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from torch import nn
import torch.nn.functional as F
from torch.utils.data.dataloader import default_collate

from skorch import NeuralNetClassifier
from skorch.dataset import Dataset
from skorch.dataset import CVSplit


'''class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X'''

class WhoWheezing(nn.Module):

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=64, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x

# pack the data into a Dataset
'''class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, i):
        Xi = self.X[i]
        yi = self.y[i]
        return self.transform(Xi, yi)'''

class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
    self.shift_pct = 0.4
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return self.transform(aug_sgram, class_id)
    #return aug_sgram, class_id

#ds = MyDataset(df1, data_path)
ds = breathDS(df, data_path)

class SliceDatasetX(Dataset):
    """Helper class that wraps a torch dataset to make it work with sklearn"""
    def __init__(self, dataset, collate_fn=default_collate):
        self.dataset = dataset
        self.collate_fn = collate_fn

        self._indices = list(range(len(self.dataset)))
        
    def __len__(self):
        return len(self.dataset)
        
    @property
    def shape(self):
        return len(self),
    
    def __getitem__(self, i):
        if isinstance(i, (int, np.integer)):
            Xb = self.transform(*self.dataset[i])[0]
            return Xb
        
        if isinstance(i, slice):
            i = self._indices[i]

        Xb = self.collate_fn([self.transform(*self.dataset[j])[0] for j in i])
        return Xb
    
   
    
params = {
    'lr': [0.001, 0.01, 0.005],
    'max_epochs': [50, 52, 54, 56, 58, 60, 62, 64, 66],
    'batch_size': [4, 8, 16, 32]
}

net = NeuralNetClassifier(
    WhoWheezing,
    max_epochs=50,
    lr=0.001,
    batch_size=16,
    iterator_train__shuffle=True,
    verbose=0,
    train_split=None
)


In [10]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [11]:
gs = GridSearchCV(net, params, refit=False, cv=5, scoring='accuracy', verbose=0)

# we have to extract the target data, otherwise sklearn will complain
y_from_ds = np.asarray([ds[i][1] for i in range(len(ds))])
# wrap the dataset into the new helper class
ds_sliceable = SliceDatasetX(ds)

gs.fit(ds_sliceable , y_from_ds)

GridSearchCV(cv=5,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.WhoWheezing'>,
),
             param_grid={'batch_size': [4, 8, 16, 32],
                         'lr': [0.001, 0.01, 0.005],
                         'max_epochs': [50, 52, 54, 56, 58, 60, 62, 64, 66]},
             refit=False, scoring='accuracy')

In [12]:
a = pd.DataFrame(gs.cv_results_)

In [13]:
print(a)

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        93.174505      0.763168         0.069235        0.000947   
1        95.479398      0.251192         0.069126        0.000968   
2        98.449119      0.771107         0.070002        0.001549   
3       100.452800      0.124774         0.069828        0.000431   
4       103.003751      0.424710         0.070314        0.000973   
5       105.721941      0.152256         0.070177        0.000664   
6       108.084597      0.319091         0.070222        0.000595   
7       111.533624      0.337624         0.071777        0.001089   
8       114.189250      0.239296         0.072173        0.001266   
9        94.996022      0.284557         0.071241        0.000905   
10       96.497409      0.376953         0.071375        0.001785   
11       98.721084      0.166927         0.070130        0.001274   
12      101.350374      0.669834         0.070257        0.000879   
13      104.553439      0.315732  

In [14]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_lr,param_max_epochs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,93.174505,0.763168,0.069235,0.000947,4,0.001,50,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 50}",0.867347,0.884354,0.823129,0.935374,0.894558,0.880952,0.036571,83
1,95.479398,0.251192,0.069126,0.000968,4,0.001,52,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 52}",0.863946,0.857143,0.870748,0.921769,0.897959,0.882313,0.024109,80
2,98.449119,0.771107,0.070002,0.001549,4,0.001,54,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 54}",0.891156,0.857143,0.826531,0.914966,0.891156,0.876190,0.030921,94
3,100.452800,0.124774,0.069828,0.000431,4,0.001,56,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 56}",0.877551,0.870748,0.846939,0.925170,0.884354,0.880952,0.025453,83
4,103.003751,0.424710,0.070314,0.000973,4,0.001,58,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 58}",0.863946,0.897959,0.816327,0.928571,0.897959,0.880952,0.038241,83
5,105.721941,0.152256,0.070177,0.000664,4,0.001,60,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 60}",0.874150,0.877551,0.816327,0.911565,0.877551,0.871429,0.030756,101
6,108.084597,0.319091,0.070222,0.000595,4,0.001,62,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 62}",0.857143,0.833333,0.860544,0.911565,0.904762,0.873469,0.029917,97
7,111.533624,0.337624,0.071777,0.001089,4,0.001,64,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 64}",0.850340,0.863946,0.840136,0.904762,0.904762,0.872789,0.027177,99
8,114.189250,0.239296,0.072173,0.001266,4,0.001,66,"{'batch_size': 4, 'lr': 0.001, 'max_epochs': 66}",0.880952,0.857143,0.819728,0.928571,0.887755,0.874830,0.035894,96
9,94.996022,0.284557,0.071241,0.000905,4,0.01,50,"{'batch_size': 4, 'lr': 0.01, 'max_epochs': 50}",0.904762,0.877551,0.877551,0.921769,0.887755,0.893878,0.017129,45
